In [2]:
# The deployment card from HF
# Not sure if the 'HF_TASK': 'text-generation' is proper for Llava
# because it is an Image Generation / Image QnA model, it'd be rather
# 'question-answering' or 
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel

sagemaker_session = sagemaker.Session()
# try:
#     role = sagemaker.get_execution_role()
# except ValueError:
#     iam = boto3.client('iam')
#     role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

role = 'arn:aws:iam::962716963657:role/service-role/AmazonSageMaker-ExecutionRole-20230922T133075'

# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID': 'liuhaotian/llava-llama-2-13b-chat-lightning-preview',
    'HF_TASK': 'text-generation'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    env=hub,
    role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.xlarge' # ec2 instance type
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/csaba/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/csaba/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/csaba/.config/sagemaker/config.yaml
-----!

In [ ]:
# Optional
!pip install sagemaker

In [7]:
from sagemaker.predictor import Predictor
from base64 import b64encode

endpoint = 'huggingface-pytorch-inference-2023-10-26-04-40-17-036'
ENCODING = "utf-8"
IMAGE_NAME = "eiffel_tower_224.png"

payload = {
    "inputs": "Describe the content of the image in great detail",
}
with open(IMAGE_NAME, 'rb') as f:
    byte_content = f.read()
    base64_bytes = b64encode(byte_content)
    base64_string = base64_bytes.decode(ENCODING)

predictor = Predictor(endpoint)
inference_response = predictor.predict(data=base64_string)
print (inference_response)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/csaba/.config/sagemaker/config.yaml


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027llava\u0027"
}
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2023-10-26-04-40-17-036 in account 962716963657 for more information.